# Qu'est-ce qu'une métaclasse ?

> « Metaclasses are deeper magic that 99% of users should never worry about. If you wonder whether you need them, you don't (the people who actually need them know with certainty that they need them, and don't need an explanation about why). »  
*Tim Peters, Python Guru*

Ce qui se traduit approximativement en : "Les métaclassent sont la magie profonde de Python dont 99% des personnes qui programment dans ce langage n'ont pas à se soucier. Si vous vous demandez ce que c'est, c'est que vous n'en avez pas besoin. Si vous en avez besoin, vous savez ce que c'est et n'avez pas besoin d'explication à leur sujet."

Merci Tim ! Nous voilà bien avancés !

Ceci dit, ce n'est pas complètement faux. Il reste assez rare de créer des métaclasses en Python. Mais elles peuvent solutionner élégamment des problèmes complexes. 

De nombreuses librairies comme Django les utilisent dans leur mécanique interne.

Alors qu'est-ce qu'une métaclasse ? C'est une classe dont les instances sont des classes.

Autrement dit, une classe est une usine qui fabrique des variables, appelées *instances de classe*. Une métaclasse est une usine qui fabrique des classes. Ou encore, une usine qui fabrique des usines.

L'intérêt est de pouvoir faire bénéficier les classes basées sur une métaclasse d'un même comportement sans avoir à le copier/coller d'une classe à l'autre.

L'héritage est fait pour cela, soulignerez-vous avec pertinence. Mais l'héritage ne résout pas tout...

## Les limites de l'héritage

Imaginons que vous souhaitiez réaliser une classe Singleton.  
Une classe singleton est une classe qui n'a qu'une seule instance d'une variable.  Si vous créez plusieurs instances, la classe retournera toujours la première créée. Et toutes vos instances auront le même **id**.

A quoi sert ce genre de classes ?

* La constante *None* est un exemple de singleton. Elle n'existe qu'en 1 seul exemplaire. C'est pour cela que dans les tests Python vous pouvez écrire:

```python
a == None or a is None
```

L'opérateur **is** qui compare les adresses fonctionne sur ce test d'égalité car toutes les instances de None ont la même adresse. Ce ne serait pas vrai pour des entiers:

```python
a = 10**16
b = 10**16
print(a == b) # True
print(a is b) # Très certainement False
```

* Les drivers qui pilotent des périphériques comme une imprimante sont souvent des objets singleton: pas besoin d'avoir 50 exemplaires du pilote en mémoire, un seul peut discuter à la fois avec l'imprimante.
* Certains logiciels comme *Word* sont des singletons, si vous le lancez plusieurs fois, c'est l'application active qui est ré-affichée.



### Comment implémenter une classe Singleton en Python

Chaque langage a ses techniques. En Java/C++ on peut faire une classe avec un constructeur privé pour empêcher qu'il soit appelé par les développeurs et les forcer à passer une méthode **get_instance** qui gère l'unique appel de constructeur de la classe.

En Python, tout est publique, cette solution n'est pas applicable.  Le site [Wikipédia présente quelques solutions](https://fr.wikipedia.org/wiki/Singleton_(patron_de_conception)) pour différents langages dont Python. 

Pour implémenter une classe Singleton nous devons l'empêcher de créer plus d'une instance.

En Python, la création d'une variable se fait dans le constructeur. 

Le constructeur n'est pas la fonction `__init__`.  
Le constructeur c'est la fonction qui crée la variable en mémoire.  
`__init__` est appelée pour initialiser les attributs de l'instance, une fois cette dernière créée. C'est la fonction [`__new__`](https://docs.python.org/3/reference/datamodel.html#object.__new__) qui alloue l'objet en mémoire.

Généralement on ne la définit jamais car Python gère la mémoire lui-même. Et c'est bien plus simple ainsi.

Mais ici nous pourrions la redéfinir pour qu'elle:

* crée une variable lors du tout premier appel et mémorise cette dernière
* retourne la variable déjà créée lors des appels suivants

L'unique instance singleton pouvant être mémorisée dans un attribut de classe.  
Ce qui donne:

In [3]:
class Singleton(object):
    
    instance_unique = None  # Attribut de classe mémorisant l'unique instance
    
    def __new__(cls, *args, **kwargs):
        """Remarquez ici, le premier paramètre n'est pas self mais la classe, car la variable 
        n'est pas encore créée - nous allons justement la créer.
        """
        
        # Si l'instance n'est pas encore créée...
        if cls.instance_unique is None:
            # On la crée en appelant la méthode __new__ du parent qui saura comment faire...
            cls.instance_unique = super().__new__(cls, *args, **kwargs)
            
        return cls.instance_unique

Il ne reste plus qu'à tester:

In [2]:
s1 = Singleton()
s2 = Singleton()

assert s1 == s2
assert s1 is s2
assert id(s1) == id(s2)
assert type(s1) == Singleton
print("Bien joué !")

Bien joué !


Mission accomplie brillament, avec l'usage de la méthode `__new__` ce qui est assez rare en Python. Cela commence à être du haut niveau.

Maintenant, imaginons que nous ayons besoin d'une seconde classe Singleton. Nous n'allons pas dupliquer la première. Héritons de cette dernière, la mécanique est déjà en place !

Pour cela je duplique (en mode notebook) la classe précédente, comme vous pourriez le faire dans un script afin de ré-initialiser `Singleton.instance_unique` à None.

In [7]:
class Singleton(object):
    
    instance_unique = None  # Attribut de classe mémorisant l'unique instance
    
    def __new__(cls, *args, **kwargs):
        """Remarquez ici, le premier paramètre n'est pas self mais la classe, car la variable 
        n'est pas encore créée - nous allons justement la créer.
        """
        
        # Si l'instance n'est pas encore créée...
        if cls.instance_unique is None:
            # On la crée en appelant la méthode __new__ du parent qui saura comment faire...
            cls.instance_unique = super().__new__(cls, *args, **kwargs)
            
        return cls.instance_unique
    
class SingletonChild(Singleton):
    pass


sc1 = SingletonChild()
sc2 = SingletonChild()

assert sc1 == sc2
assert sc2 is sc1
assert id(sc1) == id(sc2)

assert type(sc1) == SingletonChild, "Je ne suis pas du type attendu"

print("Héhé... trop facile... Quel est le besoin d'une métaclase ?")

Héhé... trop facile... Quel est le besoin d'une métaclase ?


Trop facile, ou presque...

Imaginons, que vous ayez besoin de 2 instances singleton, une pour la classe *Singleton* et une pour *SingletonChild*. Vous écririez alors ceci...

In [2]:
class Singleton(object):
    
    instance_unique = None  # Attribut de classe mémorisant l'unique instance
    
    def __new__(cls, *args, **kwargs):
        """Remarquez ici, le premier paramètre n'est pas self mais la classe, car la variable 
        n'est pas encore créée - nous allons justement la créer.
        """
        
        # Si l'instance n'est pas encore créée...
        if cls.instance_unique is None:
            # On la crée en appelant la méthode __new__ du parent qui saura comment faire...
            cls.instance_unique = super().__new__(cls, *args, **kwargs)
            
        return cls.instance_unique
    
class SingletonChild(Singleton):
    pass


# Création de l'instance singleton pour la classe Singleton
s1 = Singleton()
s2 = Singleton()

assert s1 == s2
assert s1 is s2
assert id(s1) == id(s2)
assert type(s1) == Singleton
print("Bien joué !")

# Création de l'instance singleton pour la classe SingletonChild
# ICI ça se gâte...

sc1 = SingletonChild()
sc2 = SingletonChild()

assert sc1 == sc2
assert sc2 is sc1
assert id(sc1) == id(sc2)

assert type(sc1) == SingletonChild, "Je ne suis pas du type attendu"

print("OUILLE...")

Bien joué !


AssertionError: Je ne suis pas du type attendu

In [9]:
assert isinstance(sc1, SingletonChild), "Même ainsi..."

AssertionError: Même ainsi...

In [10]:
print(type(sc1))
print(isinstance(sc1, Singleton))

<class '__main__.Singleton'>
True


In [11]:
print(sc1 == s1)

True


Et faisant hériter SingletonChild de Singleton, la classe fille bénéficie de la même fonction `__new__` mais celle-ci a déjà alloué l'instance unique pour la variable *s1* laquelle est du type Singleton. 

Du coup SingletonChild ne possède pas sa propre instance.

Il faudrait dupliquer l'attribut `instance_unique` dans SingletonChild:

In [12]:
class SingletonChild(Singleton):
    instance_unique = None
    
    
sc1 = SingletonChild()
sc2 = SingletonChild()

assert sc1 == sc2
assert sc2 is sc1
assert id(sc1) == id(sc2)
assert isinstance(sc1, SingletonChild)
print("Ok, mais on a dupliqué un attribut de classe")

Ok, mais on a dupliqué un attribut de classe


La solution est viable et presque jolie, en dehors de cette création d'instance quasiment incontournable pour chaque sous classe de Singleton.

On peut très bien vivre avec. Mais on peut aussi faire mieux en automatisant ce code avec les métaclasses.
ce qui nous permettra aussi de ne pas avoir besoin de faire hériter SingletonChild de Singleton pour qu'elle soit une classe Singleton...

## Comment sont créées les classes en Python
Avant de nous lancer dans la création de métaclasses et de classes basées sur ce modèle, rentrons plus en détails dans la compréhension du langage et de la création d'une classe.

En Python *tout est objet*.  
Une fonction est un objet, une classe est un objet...

In [24]:
def  hello():
    """Affiche bonjour"""
    print("Bonjour !")
    
print(type(hello))

<class 'function'>


In [23]:
salut = hello
salut()

Bonjour !


Que peut-on faire avec hello ?

In [25]:
print(dir(hello))

['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [27]:
print(hello.__doc__)   # la docstring est stockée ici
print(hello.__name__)  # le nom de la fonction

Affiche bonjour
hello


ET vous pourriez aussi ajouter des attributs sur la fonction, comme un compteur d'appels... Mais ce n'est pas l'objet de cet article.

Les types Python comme *int*, *str*, *list* sont aussi des objets. Ce sont des classes qui sont des instances de métaclasses.

Quel est donc le type de **int**, autrement dit, la métaclasse qui a créé la classe *int* ?   
Est-ce **object** ?   
Non, *object* est la classe parent de *int*, ce n'est pas la classe qui a créé la classe *int*. C'est la classe dont hérite *int*, tout comme *SingletonChild* hérite de *Singleton*. Mais *Singleton* n'a pas créé *SingletonChild*. *Singleton* a créé les variables **s1** et **s2**.

In [28]:
print(type(int))

<class 'type'>


**Type** est le type de *int*. **type** n'est pas juste une fonction qui affiche le type d'une variable, c'est une classe qui crée des classes, c'est une **métaclasse** !

Mais qui a créé **type** ?

Python peut-il répondre à la question du pourquoi du comment de l'univers et de tout ce qui est vivant ?

La réponse est-elle *Guido Van Rossum* ou *42* ?   
N'attendons plus pour savoir, la réponse au Saint-Graal de l'existence des objets est là, tout prêt !  
Posons la question !

In [29]:
print(type(type))  # Alors ?

<class 'type'>


C'est *type* ! *type* s'est créée elle-même ! C'est la classe toute puissante !

La classe *type* peut donc servir à fabriquer des variables, qui sont des classes !

Si on lui passe un seul paramètre, elle retourne le type de cette variable, sinon elle construit une classe.

Voici les paramètres acceptés dans sa seconde utilisation:

* Le nom de la classe créée
* Un tuple qui contient la liste des parents
* Un dictionnaire qui contient les attributs et méthodes de la classe

Si vous écrivez ce code:
    


In [14]:
class A(object):
    pass

class B(object):
    pass

MaClasse = type("MaClasse",
                (A, B),
                {"color" : 'blue', 
                 'hello' : lambda self: 'Hello man!'}
                )
mc = MaClasse()
print(type(mc))
print(MaClasse.color)
print(mc.hello())
print(MaClasse.__mro__)

<class '__main__.MaClasse'>
blue
Hello man!
(<class '__main__.MaClasse'>, <class '__main__.A'>, <class '__main__.B'>, <class 'object'>)


Il est équivalent à celui-ci:

In [15]:
class A(object):
    pass

class B(object):
    pass

class MaClasse(A, B):
    color = "blue"
    
    def hello(self):
        return "Hello man!"

mc = MaClasse()
print(type(mc))
print(MaClasse.color)
print(mc.hello())
print(MaClasse.__mro__)

<class '__main__.MaClasse'>
blue
Hello man!
(<class '__main__.MaClasse'>, <class '__main__.A'>, <class '__main__.B'>, <class 'object'>)


C'est assez bluffant au début. Mais pas très pratique à mettre en oeuvre: définir des méthodes au travers d'un dictionnaire ce n'est pas très heureux.

Pour résumer, quand vous écrivez:

```python
class MaClasse(object):
    pass
```

Python exécute:

```python
MaClasse = type("MaClasse", (object, ), {} )
```

Ce qui reste valide, mais il est plus confortable d'utiliser le mot clef **class**.

## Comment crée-t-on une métaclasse ?

Pour cela il suffit de créer une classe qui hérite de **type**.


```python
class MaMetaClasse(type):
    pass
```

Une métaclasse est une classe qui crée des classes, comme type, nous pourrions donc écrire:

```python
MaClasse = MaMetaClasse()
mon_object = MaClasse()
```

Ne confondez pas :

* Le type d'une variable: C'est le nom de la classe qui a créé la variable
* Le parent d'une classe : C'est la classe dont hérite votre classe, que vous pouvez consulter avec : `MyClass .__base__`

In [19]:
class MaMetaClasse(type):
    pass

MaClasse = MaMetaClasse("MaClasse", (object,), {})
mon_object = MaClasse()

print(type(MaClasse))
print(type(mon_object))
print(MaClasse.__base__)

<class '__main__.MaMetaClasse'>
<class '__main__.MaClasse'>
<class 'object'>


Mais cette syntaxe n'est pas très heureuse.  
La [PEP3115](https://www.python.org/dev/peps/pep-3115/) présente la syntaxe officielle pour définir une classe basée sur une métaclasse:

In [21]:
class MaMetaClasse(type):
    pass

class MaClasse(object, metaclass=MaMetaClasse):
    pass

print(type(MaClasse))
print(type(mon_object))
print(MaClasse.__base__)

<class '__main__.MaMetaClasse'>
<class '__main__.MaClasse'>
<class 'object'>


## Comment implémenter la classe Singleton avec une MetaClasse ?


L'idée est la suivante, si la classe doit posséder un attribut `instance_unique`, il suffit de l'initialiser dans la métaclasse. Ainsi chaque classe basée sur la métaclasse possèdera cet attribut sans qu'il y ait besoin de copier/coller ce code.

### Précisions sur la mécanique de création des objets en Python

Quand on écrit:

```python
ma_variable = MaClasse()
```

* Python alloue l'objet en mémoire avec la fonction `MaClasse.__new__`
* Puis il initialise l'objet avec la fonction `MaClasse.__init__`  
  La fonction `__init__` permet d'affecter des attributs à l'objet créé. 

Mais vous savez aussi que l'on peut appeler un objet comme une fonction si la classe de l'objet définit la méthode spéciale `__call__`.

Exemple:





In [23]:
class Vehicule:
    
    def __call__(self, *args, **kwargs):
        print("En voiture", *args)
        
v = Vehicule()
v("Paul", "Pierre", "Jacqueline")
        
        

En voiture Paul Pierre Jacqueline


Pour créer une variable, vous appelez la classe comme une fonction:  

`variable = MaClasse()` 

Ceci appelle donc la méthode `__call__` de la métaclasse en lui passant la classe en premier paramètre:

* qui crée la variable avec `MaClasse.__new__()`
* puis initialise la variable avec `MaClasse.__init__()`
* puis retourne la variable 

Illustrons ceci avec un exemple concret...


In [31]:
print('Création de la métaclasse')
class MetaClasse(type):
    
    def __new__(cls, *args, **kwargs):
        print("Dans MetaClasse.__new__")
        return super().__new__(cls, *args, **kwargs)
    
    def __init__(cls, *args, **kwargs):
        print("Dans MetaClasse.__init__", cls.__name__)
        super().__init__(*args, **kwargs)
    
    def __call__(cls, *args, **kwargs):
        print("Dans MetaClasse.__call__", cls.__name__)
        return super().__call__(*args, **kwargs)
        
    
print("Création de la classe")
class MaClasse(object, metaclass=MetaClasse):

    def __new__(cls, *args, **kwargs):
        print("MaClasse.__new__")
        return super().__new__(cls, *args, **kwargs)

    def __init__(self, *args, **kwargs):
        print("MaClasse.__init__")
        super().__init__(*args, **kwargs)

    def __call__(self, *args, **kwargs):
        print("MaClasse.__call__")
        print(*args, **kwargs)

print("Création de la variable")
i = MaClasse()
print("Appel de la variable comme une fonction")
i("Mais quelle belle mécanique...")

Création de la métaclasse
Création de la classe
Dans MetaClasse.__new__
Dans MetaClasse.__init__ MaClasse
Création de la variable
Dans MetaClasse.__call__ MaClasse
MaClasse.__new__
MaClasse.__init__
Appel de la variable comme une fonction
MaClasse.__call__
Mais quelle belle mécanique...


**Notez** l'usage de *cls* dans l'appel à *super()* pour la méthode `__new__` et sans *cls* pour les autres comme `__init__`.  
En effet, la variable n'est pas encore créée quand on appelle `__new__` il faut donc passer l'attribut cls qui ne représente pas l'instance.

Nous pouvons maintenant écrire une métaclasse **MetaSingleton** :

* Lorsque nous créerons une classe basée sur cette métaclasse cela appellera ses méthodes `__new__` et `__init__`
* Lorsque nous créerons une instance de classe basée sur la métaclasse MetaSingleton, cela appellera ̀`MetaSingleton.__call__(Classe)`  
C'est là que vous devrez tester l'existence de l'instance unique
* Dans la méthode `__init__` de la métaclasse, initialisez l'attribut statique de la classe, ce sera plus simple

In [34]:
class MetaSingleton(type):
    
    def __init__(cls, *args, **kwargs):
        """cls est la classe créée par la métaclasse"""
        # ajout de l'attribut de classe instance unique
        cls.instance_unique = None
        
        
    def __call__(cls, *args, **kwargs):
        
        # On crée l'instance unique existe si elle n'existe pas déjà
        if cls.instance_unique is None:
            cls.instance_unique = super().__call__(*args, **kwargs)
            
        return cls.instance_unique
    
    
class Singleton(object, metaclass=MetaSingleton):
    pass


class SingletonChild(Singleton):
    pass


# Création de l'instance singleton pour la classe Singleton
s1 = Singleton()
s2 = Singleton()

assert s1 == s2
assert s1 is s2
assert id(s1) == id(s2)
assert type(s1) == Singleton

# cette fois tout ira bien !

sc1 = SingletonChild()
sc2 = SingletonChild()

assert sc1 == sc2
assert sc2 is sc1
assert id(sc1) == id(sc2)

assert type(sc1) == SingletonChild, "Je ne suis pas du type attendu"

print("Extra génial")


Extra génial


Vous voilà prêt(e) pour bâtir de grandes fondations...  
Mais très sincèrement, on ne crée que rarement des métaclasses, sauf si l'on développe des frameworks sophistiqués comme l'ORM de Django. 

A défaut d'être appliqué professionnellement, ce tutoriel aura essayé de vous apporter la compréhension de la mécanique interne de Python.

Et comme le dit *Tim Peters* : **It's deep magic !**




Pour vous faire la main:

* essayez d'implémenter une métaclasse qui gère un compteur d'objets créés par la classe.
* puis qui le décrémente quand ils sont détruits...

Hum, est-ce possible ?

# Pour aller plus loin

* Un très bon fil de discussion Stackoverflow sur le sujet:  
  http://stackoverflow.com/questions/100003/what-is-a-metaclass-in-python 
* Un autre [tutoriel intéressant sur les métaclasses](https://blog.ionelmc.ro/2015/02/09/understanding-python-metaclasses/)
